In [1]:
#r "nuget:VegaLite.NET"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package VegaLite.NET version 1.3.22

In [1]:
using VegaLite;
using VegaLite.Schema;

using Newtonsoft.Json;
using Newtonsoft.Json.Serialization;
using System.Net.Http;
using System.IO;

In [1]:
public struct SplomData
{
    [JsonProperty("Iteration",
                  NamingStrategyType = typeof(DefaultNamingStrategy))]
    public int Iteration { get; set; }

    [JsonProperty("SwarmIndex",
                  NamingStrategyType = typeof(DefaultNamingStrategy))]
    public int SwarmIndex { get; set; }

    [JsonProperty("Particle",
                  NamingStrategyType = typeof(DefaultNamingStrategy))]
    public int Particle { get; set; }

    [JsonProperty("km",
                  NamingStrategyType = typeof(DefaultNamingStrategy))]
    public float km { get; set; }

    [JsonProperty("kF",
                  NamingStrategyType = typeof(DefaultNamingStrategy))]
    public float kF { get; set; }

    [JsonProperty("kf",
                  NamingStrategyType = typeof(DefaultNamingStrategy))]
    public float kf { get; set; }

    [JsonProperty("ye",
                  NamingStrategyType = typeof(DefaultNamingStrategy))]
    public float ye { get; set; }

    [JsonProperty("LF",
                  NamingStrategyType = typeof(DefaultNamingStrategy))]
    public float LF { get; set; }

    [JsonProperty("Lf",
                  NamingStrategyType = typeof(DefaultNamingStrategy))]
    public float Lf { get; set; }

    public SplomData(int    iteration,
                     int    swarmIndex,
                     int    particle,
                     double km,
                     double kF,
                     double kf,
                     double ye,
                     double lF,
                     double lf)
    {
        Iteration  = iteration;
        SwarmIndex = swarmIndex;
        Particle   = particle;
        this.km    = (float)km;
        this.kF    = (float)kF;
        this.kf    = (float)kf;
        this.ye    = (float)ye;
        LF         = (float)lF;
        Lf         = (float)lf;
    }

    public SplomData(string data)
    {
        string[] columns = data.Split(new char[]
                                      {
                                          ','
                                      },
                                      StringSplitOptions.RemoveEmptyEntries);

        int index = 0;

        Iteration  = int.Parse(columns[index++]);
        SwarmIndex = int.Parse(columns[index++]);
        Particle   = int.Parse(columns[index++]);
        km         = float.Parse(columns[index++]);
        index++;
        kF = float.Parse(columns[index++]);
        index++;
        kf = float.Parse(columns[index++]);
        index++;
        ye = float.Parse(columns[index++]);
        index++;
        LF = float.Parse(columns[index++]);
        index++;
        Lf = float.Parse(columns[index]);
    }
}

In [1]:
string[] pso_lines = File.ReadAllLines("Data/PSO.arrow");

In [1]:
int line_count = 10; //pso_lines.Length

List<SplomData> pso_data_list = new List<SplomData>(line_count);

for (int i = 0; i < line_count; ++i)
{
    pso_data_list.Add(new SplomData(pso_lines[i + 1]));
}

SplomData[] pso_data = pso_data_list.ToArray();

In [1]:
Specification specification = new Specification
{
    Repeat = new RepeatMapping()
    {
        Row = new List<string>()
        {
            "km",
            "kF",
            "kf",
            "ye",
            "LF",
            "Lf"
        },
        Column = new List<string>()
        {
            "km",
            "kF",
            "kf",
            "ye",
            "LF",
            "Lf"
        }
    },
    Spec = new SpecClass()
    {
        DataSource = new DataSource()
        {
            Name = "pso_data"
        },
        Transform = new List<Transform>()
        {
            new Transform()
            {
                Filter = new Predicate()
                {
                    Selection = "IterationSelection"
                }
            }
        },
        Mark = BoxPlot.Circle,
        Encoding = new Encoding()
        {
            X = new XClass()
            {
                Type = StandardType.Quantitative,
                Field = new RepeatRef()
                {
                    Repeat = RepeatEnum.Column
                }
            },
            Y = new YClass()
            {
                Type = StandardType.Quantitative,
                Field = new RepeatRef()
                {
                    Repeat = RepeatEnum.Row
                }
            },
            Color = new DefWithConditionMarkPropFieldDefGradientStringNull()
            {
                Type = StandardType.Nominal, Field = "SwarmIndex",
                Scale = new Scale()
                {
                    Scheme = ColorThemes.Cyclical.Rainbow
                }
            },
            //Opacity = new DefWithConditionMarkPropFieldDefNumber()
            //{
            //    Condition = new ConditionalDef()
            //    {
            //        Selection = "Select_Swarm", Value = 1.0
            //    },
            //    Value = 0.2
            //}
        },
        Selection = new Dictionary<string, SelectionDef>()
        {
            {
                "IterationSelection", new SelectionDef()
                {
                    Type = SelectionDefType.Single,
                    Fields = new List<string>()
                    {
                        "Iteration"
                    },
                    Init = new Dictionary<string, InitValue>()
                    {
                        {
                            "Iteration", 0
                        }
                    },
                    Bind = new Dictionary<string, AnyStream>()
                    {
                        {
                            "Iteration", new AnyBinding()
                            {
                                Input = "range",
                                Min   = 0.0,
                                Max   = 99.0
                            }
                        }
                    }
                }
            }
            //{
            //    "Select_Swarm", new SelectionDef()
            //    {
            //        Type = SelectionDefType.Multi,
            //        Fields = new List<string>()
            //        {
            //            "SwarmIndex"
            //        },
            //        Bind = SelectionLegendBinding.Legend
            //    }
            //}
        }
    }
};

In [1]:
Specification specification = new Specification
{
    Repeat = new RepeatMapping()
    {
        Row = new List<string>()
        {
            "kmatrix",
            "kFracture",
            "kNaturalFracture",
            "ye",
            "LFracture",
            "LNaturalFracture"
        },
        Column = new List<string>()
        {
            "kmatrix",
            "kFracture",
            "kNaturalFracture",
            "ye",
            "LFracture",
            "LNaturalFracture"
        }
    },
    Spec = new SpecClass()
    {
        DataSource = new DataSource()
        {
            Name = "pso_data",
            Url = "Data/PSO.arrow",
            Format = new DataFormat()
            {
                Type = "arrow"
            }            
        },
        Mark = BoxPlot.Circle,
        Encoding = new Encoding()
        {
            X = new XClass()
            {
                Type = StandardType.Quantitative,
                Field = new RepeatRef()
                {
                    Repeat = RepeatEnum.Column
                }
            },
            Y = new YClass()
            {
                Type = StandardType.Quantitative,
                Field = new RepeatRef()
                {
                    Repeat = RepeatEnum.Row
                }
            },
            Color = new DefWithConditionMarkPropFieldDefGradientStringNull()
            {
                Type = StandardType.Nominal,
                Field = "SwarmIndex",
                Scale = new Scale()
                {
                    Scheme = ColorThemes.Cyclical.Rainbow
                }
            }
        }
    }
};

Chart chart = new Chart("Splom", specification, 1906, 450);

In [1]:
chart

In [1]:
//string output = JsonConvert.SerializeObject(pso_data, Formatting.Indented);

Formatter<Chart<VegaLite.Test.SplomData>>.Register((chart,
                                        writer) =>
                                       {
                                           writer.Write(chart.GetHtml(pso_data.Length));
                                       },
                                       HtmlFormatter.MimeType);

In [1]:
Chart<SplomData> chart = new Chart<SplomData>("Splom", specification, 1906, 450);

In [1]:
chart

In [1]:
chart.GetHtmlContent(pso_data.Length).ToString()

In [1]:
//#!js
//interactive.csharp.getVariable("pso_data").then(result => { console.log(result); });

In [1]:
//string output = JsonConvert.SerializeObject(pso_data, Formatting.Indented);

In [1]:
//Formatter<Chart<SplomData>>.Register((chart, writer) =>
//{
//   writer.Write(chart.GetHtml(pso_data.Length));
//}, HtmlFormatter.MimeType);

In [1]:
//#!js
const scripts = document.getElementsByTagName("script");
var dotnet_script = "";

for (let script of scripts) {
    let status = script.getAttribute("data-requiremodule");
    if (status === "dotnet-interactive/dotnet-interactive") {
        dotnet_script = script.src;
        break;
    }
}

const address = dotnet_script.substring(0, dotnet_script.length - 31);

console.log(address);